In [1]:
import os, sys 
%pwd

'/config/workspace/notebook'

In [2]:
os.chdir("../")
%pwd

'/config/workspace'

In [3]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_url: str 
    local_data_file: Path 
    unzip_dir: Path 

In [4]:
from accident_severity.constants import * 
from accident_severity.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config 

In [6]:
import os, sys 
import urllib.request as request 
import zipfile
from accident_severity.logging import logger
from accident_severity.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} downloaded succesfully with following headers: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[{asctime}s: INFO: common: yaml file: config/config.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: params.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: schema.yaml loaded successfully]
[{asctime}s: INFO: common: created directory at: artifacts]
[{asctime}s: INFO: common: created directory at: artifacts/data_ingestion]
[{asctime}s: INFO: 2586585474: artifacts/data_ingestion/RTA_data.zip downloaded succesfully with following headers: 
Connection: close
Content-Length: 280014
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "7bb41a6c214ca8066f1caa9b2f9bd23da912734b54c951dca42e9c991eb9d3eb"
Last-Modified: Thu, 07 Sep 2023 14:29:30 GMT
Strict-Transport-Security: max-age=31557600
timing-allow-origin: https://github.com
X-Content-Type-Options: nosniff
X-Frame-Options: deny
x-github-tenant: 
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C12C:347CEE:70C1E:9971B:64F9DE3D
Accept-Ranges: bytes
Date: